In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_api_key

# Access gmaps with API
gmaps.configure(g_api_key)

#base url needed for the api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [25]:
# csv path
csv = "../output_data/cities.csv"

# read csv
df = pd.read_csv(csv)

# show data frame
df.head()
len(df)

574

In [26]:
# Customize the size of the map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

locations = df[['Latitude', 'Longitude']]
weights = df['Humidity']
fig = gmaps.figure(layout = figure_layout)

# Add heatmap layer over map
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [27]:
#Locate cities from csv where tempature between 70 and 80/ wind Speed 10 or less/ cloudiness = 0
#use .loc function to place in another dataframe 
#name new data frame hotel_df
hotel_df = df.loc[(df["Max Temperature"] >= 70) & (df["Max Temperature"] <= 80) &
                    (df["Wind Speed"] <= 10) & (df["Cloudiness"] == 0)]

hotel_df


,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
170,Tomatlán,MX,2020-06-16 00:05:05,19.93,-105.25,0,73,76.78,2.21
244,Maţāy,EG,2020-06-16 00:05:26,28.42,30.78,0,47,70.45,9.08
298,Manacapuru,BR,2020-06-16 00:05:35,-3.30,-60.62,0,94,75.20,2.59
422,Sakakah,SA,2020-06-16 00:01:22,29.97,40.21,0,27,75.20,8.05
485,San Patricio,MX,2020-06-16 00:01:06,19.22,-104.70,0,73,78.53,5.03
532,Borūjerd,IR,2020-06-16 00:06:16,33.90,48.75,0,26,71.60,6.93


In [32]:

# set parameters
params = {
    "key": g_api_key,
    "radius": 5000,
    "keyword": "hotel",
}

# loop through hotel_df and take lat/long
for index, row in hotel_df.iterrows():
    

    lat = row['Latitude']
    lng = row['Longitude']

    # update address key value
    params['location'] = f"{lat},{lng}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
        
    # convert to json
    cities_lat_lng = cities_lat_lng.json()
        
    try:
        # fill hotel data frame
        hotel_df.loc[index, 'Hotel Name'] = cities_lat_lng['results'][0]['name']
        
    except:
        print(f"Skip row {index}")

# drop rows that were skipped and print new data frame
hotel_df = hotel_df.drop(244)
hotel_df

Skip row 244


,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed,Hotel Name
170,Tomatlán,MX,2020-06-16 00:05:05,19.93,-105.25,0,73,76.78,2.21,Hotel Misión San Miguel
298,Manacapuru,BR,2020-06-16 00:05:35,-3.30,-60.62,0,94,75.20,2.59,Novo Hotel
422,Sakakah,SA,2020-06-16 00:01:22,29.97,40.21,0,27,75.20,8.05,Raoum Inn Hotel
485,San Patricio,MX,2020-06-16 00:01:06,19.22,-104.70,0,73,78.53,5.03,La Paloma Oceanfront Retreat
532,Borūjerd,IR,2020-06-16 00:06:16,33.90,48.75,0,26,71.60,6.93,Hotel khatam


In [29]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

locations = df[['Latitude', 'Longitude']]
weights = df['Humidity']
fig = gmaps.figure(layout = figure_layout)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

# Assign the marker layer to a variable
hotel_locations = hotel_df[['Latitude', 'Longitude']]
markers = gmaps.marker_layer(hotel_locations)

# Add the layer to the map
fig.add_layer(markers)

In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [31]:
# Add markers to heatmap
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Print map
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…